In [14]:
import requests
import urllib.request
import json

from SPARQLWrapper import SPARQLWrapper, JSON


In [15]:
# Fonctions utiles

# Pour enlever la qualificatif entre () dans un nom provenant de DBpedia
def traiter_nom(chaine):
    c = chaine.split(' (')
    return c[0]


# DBpedia français

In [16]:
# Recherche dans DBpedia français
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
  PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT DISTINCT ?s ?label ?naissance
  WHERE {
    ?s dcterms:subject ?cat .
    ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_québécois> .
  
    ?s  rdfs:label ?label . 
    OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
    FILTER (lang(?label) = "fr")
  } 
""")

sparql.setReturnFormat(JSON)
results_dbpedia_fr = sparql.query().convert()

In [17]:
filtre = ["Liste d'écrivains québécois par ordre alphabétique",
           "Association des écrivaines et des écrivains québécois pour la jeunesse",
           "Liste de poètes québécois",
           "Liste d'auteurs québécois par ordre chronologique",
           "Liste d'auteurs québécois par ordre chronologique de décès"]


# On récupère les données extraites par la requête SPARQL
ecrivains_dbpedia_fr = set([(b['s']['value'],
                         traiter_nom(b['label']['value']), 
                         b['naissance']['value'][:4] if 'naissance' in b else 'X') 
                        for b in results_dbpedia_fr['results']['bindings']])


# On retire ceux qui ne correspondent pas à des écrivains
ecrivains_dbpedia_fr = [(uri,nom,naiss) for (uri,nom,naiss) in ecrivains_dbpedia_fr if nom not in filtre]

print(len(ecrivains_dbpedia_fr), 'écrivains du Québec trouvés dans DBpedia français')
      
file_writers = open("../Data/DBpedia/ecrivains_dbpedia_fr.txt","w")
for e in ecrivains_dbpedia_fr:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()



1259 écrivains du Québec trouvés dans DBpedia français


### Recherche des auteurs dans DBpedia anglais

Plusieurs auteurs se retrouvent de DBpedia anglais. Même, certains sont présents seulement dans DBpedia anglais.
Pour les retrouver, on pourrait chercher ceux qui sont liés à la catégorie Writers_from_Quebec, ou une de ses sous-catégories. Mais cela couvrerait trop large, puisqu'il y a des sous-catégories, comme "McGill University alumni", qui ne représentent pas nécessairement des écrivains. Il faut donc lister toutes les sous-catégories pertinentes. Si on essaie d'utiliser toute la hiérarchie de Writers_from_Quebec et filtrer par la suite les instances du sous-arrbre dominé par la catégorie Academics_in_Quebec, ça ne marche pas, à cause d'un problème de mémoire sur le serveur.

À noter que l'on retrouve plus systématiquement les dates de naissance dans DBpedia anglais.
    
    

In [18]:

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   SELECT DISTINCT ?s ?label ?naissance  
   WHERE {
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Montreal> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec_City>}

     ?s rdfs:label ?label . 
     OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
     
     FILTER (lang(?label) IN ("fr", "en"))
    } 
""")
sparql.setReturnFormat(JSON)
results_dbpedia_en = sparql.query().convert()



In [19]:
ecrivains_dbpedia_en = set([(b['s']['value'],
                             traiter_nom(b['label']['value']),
                             b['naissance']['value'][:4] if 'naissance' in b else 'X')
                            for b in results_dbpedia_en['results']['bindings']])

print(len(ecrivains_dbpedia_en), 'écrivains du Québec trouvés dans DBpedia anglais')



file_writers = open("../Data/DBpedia/ecrivains_dbpedia_en.txt","w")
for e in ecrivains_dbpedia_en:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()
    
    
    

784 écrivains du Québec trouvés dans DBpedia anglais


Deux écrivains sont dédoublés parce qu'ils ont deux dates de naissances différentes (erreurs dans DBpedia anglais)
* Victor Barbeau
* Angeline Hango

À noter qu'il y a deux écrivains qui ont deux homonymes dans DBpedia français et qui se trouvent aussi dans DBpedia anglais:
* Pierre Morency
* Louis Émond
